## 2 The Data

In [42]:

import wrds
import pandas as pd
import datetime
import numpy as np


In [43]:
# Connect to WRDS
db = wrds.Connection()
db.create_pgpass_file()

WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [44]:
#---------------------------------------------
# WRDS Monthly World Indices
#---------------------------------------------

df = db.raw_sql("""
    SELECT * 
    FROM wrdsapps_windices.mwcountryreturns
    WHERE date BETWEEN '2002-01-01' AND '2024-12-31'
    AND country IN ('AUSTRALIA', 'FRANCE', 'GERMANY', 'JAPAN', 'SWITZERLAND', 'UNITED KINGDOM')
""")

df = df[["date", "country", "mportret", "currency"]]


#---------------------------------------------
# Risk Free Rate 
#---------------------------------------------
rf=db.raw_sql("""select  mcaldt,tmytm 
           from crsp.tfz_mth_rf            
            where kytreasnox = 2000001 
           and mcaldt>='2002-01-01'
            and mcaldt<='2024-12-31'""", date_cols=['mcaldt'])
rf['tmytm']= np.exp(rf['tmytm']/12/100)-1
rf=rf.rename(columns={ "mcaldt": "date","tmytm": "rf"})

#---------------------------------------------
# Value Weighted Index Returns US
#---------------------------------------------
rm=db.raw_sql("""select  date,vwretd from crsp.msi 
                where date>='2002-01-01' and date<='2024-12-31'
                """,date_cols=['date'])
rm = rm.rename(columns={'vwretd':'rm'})


#---------------------------------------------
# Convert date formats
#---------------------------------------------
rm['date'] = pd.to_datetime(rm['date'], format='%Y-%m-%d')
rf['date'] = pd.to_datetime(rf['date'], format='%Y-%m-%d')
rm['date'] = pd.to_datetime(rm['date']).dt.to_period('M').dt.to_timestamp()
rf['date'] = pd.to_datetime(rf['date']).dt.to_period('M').dt.to_timestamp()

#---------------------------------------------
# FX Rates
#---------------------------------------------
JPUS = pd.read_csv('EXJPUS.csv', sep=',')
SZUS = pd.read_csv('EXSZUS.csv', sep=',')
USEU = pd.read_csv('EXUSEU.csv', sep=',')
USAL = pd.read_csv('EXUSAL.csv', sep=',')
USUK = pd.read_csv('EXUSUK.csv', sep=',')

megre = pd.merge(SZUS, JPUS, on='observation_date', how='outer')
megre = pd.merge(megre, USEU, on='observation_date', how='outer')
megre = pd.merge(megre, USAL, on='observation_date', how='outer')
fx = pd.merge(megre, USUK, on='observation_date', how='outer')
fx = fx[(fx['observation_date'] >= '2002-02-01') & (fx['observation_date'] <= '2025-01-31')]
fx['EXSZUS'] = 1/fx['EXSZUS']
fx['EXJPUS'] = 1/fx['EXJPUS']
fx = fx.rename(columns={
    'observation_date': 'date',
    'EXSZUS': 'CHF',
    'EXJPUS': 'JPY',
    'EXUSEU': 'EUR',
    'EXUSAL': 'AUD',
    'EXUSUK': 'GBP'
})

fx['date'] = pd.to_datetime(fx['date'], format='%Y-%m-%d')
fx['date'] += datetime.timedelta(days=-1) # Adjusting to the last day of the month

# ---------------------------------------------
# Interbank Rates 3M
# ---------------------------------------------
rate_au = pd.read_csv("IR3TIB01AUM156N.csv", sep=',')
rate_ch = pd.read_csv("IR3TIB01CHM156N.csv", sep=',')
rate_de = pd.read_csv("IR3TIB01EZM156N.csv", sep=',')
rate_fr = pd.read_csv("IR3TIB01EZM156N.csv", sep=',')
rate_jp = pd.read_csv("IR3TIB01JPM156N.csv", sep=',')
rate_uk = pd.read_csv("IR3TIB01GBM156N.csv", sep=',')
rate_us = pd.read_csv("IR3TIB01USM156N.csv", sep=',')

rates = pd.merge(rate_au, rate_ch, on='observation_date', how='outer')
rates = pd.merge(rates, rate_de, on='observation_date', how='outer')
rates = pd.merge(rates, rate_fr, on='observation_date', how='outer')
rates = pd.merge(rates, rate_jp, on='observation_date', how='outer')
rates = pd.merge(rates, rate_uk, on='observation_date', how='outer')
rates = pd.merge(rates, rate_us, on='observation_date', how='outer')
rates = rates[(rates['observation_date'] >= '2002-02-01') & (rates['observation_date'] <= '2025-01-31')]
rates = rates.rename(columns={
    'observation_date': 'date',
    'IR3TIB01AUM156N': 'AUSTRALIA',
    'IR3TIB01CHM156N': 'SWITZERLAND',
    'IR3TIB01EZM156N_x': 'GERMANY',
    'IR3TIB01EZM156N_y': 'FRANCE',
    'IR3TIB01JPM156N': 'JAPAN',
    'IR3TIB01GBM156N': 'UNITED KINGDOM',
    'IR3TIB01USM156N': 'UNITED STATES'
})

#divide by 100 to get the rates in percentage
rates[['AUSTRALIA', 'SWITZERLAND', 'GERMANY', 'FRANCE', 'JAPAN', 'UNITED KINGDOM', 'UNITED STATES']] = rates[['AUSTRALIA', 'SWITZERLAND', 'GERMANY', 'FRANCE', 'JAPAN', 'UNITED KINGDOM', 'UNITED STATES']] / 100 /12

rates['date'] = pd.to_datetime(rates['date'], format='%Y-%m-%d')
rates['date'] += datetime.timedelta(days=-1)
print(rates.head())
#For Japan we miss 3 observations (for Japan we consider from 04/2002 - based on ED)

#---------------------------------------------
# Adjusting the dates
#---------------------------------------------
df['date'] = pd.to_datetime(df['date']).dt.to_period('M').dt.to_timestamp()
fx['date'] = pd.to_datetime(fx['date']).dt.to_period('M').dt.to_timestamp()
rates['date'] = pd.to_datetime(rates['date']).dt.to_period('M').dt.to_timestamp()

df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
fx['date'] = pd.to_datetime(fx['date'], format='%Y-%m-%d')
rates['date'] = pd.to_datetime(rates['date'], format='%Y-%m-%d')

df = pd.merge(df, fx, on='date',  how='left')
df['fx'] = df.apply(lambda row: row[row['currency']], axis=1)
df = df.drop(columns=['CHF', 'JPY', 'EUR', 'AUD', 'GBP'])

df = pd.merge(df, rf, on='date', how='left')
df = pd.merge(df, rm, on='date', how='left')

df = pd.merge(df, rates, on='date',  how='left')
df['rates'] = df.apply(lambda row: row[row['country']], axis=1)
df = df.drop(columns=['AUSTRALIA', 'SWITZERLAND', 'GERMANY', 'FRANCE', 'JAPAN', 'UNITED KINGDOM', 'UNITED STATES'])

          date  AUSTRALIA  SWITZERLAND   GERMANY    FRANCE     JAPAN  \
452 2002-01-31   0.003592     0.001306  0.002798  0.002798       NaN   
453 2002-02-28   0.003717     0.001337  0.002826  0.002826       NaN   
454 2002-03-31   0.003825     0.001217  0.002839  0.002839  0.000083   
455 2002-04-30   0.004033     0.000970  0.002889  0.002889  0.000067   
456 2002-05-31   0.004225     0.000958  0.002887  0.002887  0.000075   

     UNITED KINGDOM  UNITED STATES  
452        0.003377       0.001517  
453        0.003442       0.001592  
454        0.003478       0.001558  
455        0.003461       0.001517  
456        0.003481       0.001508  


## EX 3 A

In [45]:
df['fx_t+1'] = df['fx'].shift(-1)
df['returns_USD'] = (1 + df['mportret']) * (1 + (df['fx_t+1'] - df['fx'])/df['fx']) - 1
df['rates_USD'] = (1 + df['rates']) * (1 + (df['fx_t+1'] - df['fx'])/df['fx']) - 1
display(df.head())

,date,country,mportret,currency,fx,rf,rm,rates,fx_t+1,returns_USD,rates_USD
0,2002-01-01,AUSTRALIA,0.027358,AUD,0.5128,0.001409,-0.015966,0.003592,0.5256,0.053002,0.028642
1,2002-02-01,AUSTRALIA,-0.001832,AUD,0.5256,0.001445,-0.0217,0.003717,0.5352,0.016399,0.022049
2,2002-03-01,AUSTRALIA,0.005199,AUD,0.5352,0.001426,0.044698,0.003825,0.5498,0.03262,0.031209
3,2002-04-01,AUSTRALIA,-0.014302,AUD,0.5498,0.001462,-0.0496,0.004033,0.5682,0.018686,0.037635
4,2002-05-01,AUSTRALIA,-0.000204,AUD,0.5682,0.001409,-0.01051,0.004225,0.5538,-0.025542,-0.021225


## 3 B

In [46]:
# Calculate currency-hedged component
df['X'] = (df['fx_t+1'] / df['fx']) * (1 + df['rates']) - (1 + df['rf'])

# Calculate hedged index return
df['hedged_return'] = df['returns_USD'] - df['X']
display(df.head())

,date,country,mportret,currency,fx,rf,rm,rates,fx_t+1,returns_USD,rates_USD,X,hedged_return
0,2002-01-01,AUSTRALIA,0.027358,AUD,0.5128,0.001409,-0.015966,0.003592,0.5256,0.053002,0.028642,0.027233,0.025768
1,2002-02-01,AUSTRALIA,-0.001832,AUD,0.5256,0.001445,-0.0217,0.003717,0.5352,0.016399,0.022049,0.020604,-0.004205
2,2002-03-01,AUSTRALIA,0.005199,AUD,0.5352,0.001426,0.044698,0.003825,0.5498,0.03262,0.031209,0.029783,0.002837
3,2002-04-01,AUSTRALIA,-0.014302,AUD,0.5498,0.001462,-0.0496,0.004033,0.5682,0.018686,0.037635,0.036173,-0.017487
4,2002-05-01,AUSTRALIA,-0.000204,AUD,0.5682,0.001409,-0.01051,0.004225,0.5538,-0.025542,-0.021225,-0.022634,-0.002908


## 3C

In [47]:
#-----------------------------------------------
# Unhedged Index Returns
#-----------------------------------------------

us_only = df[df["country"] == "AUSTRALIA"].copy()
us_only["country"] = "USA"
us_only["returns_USD"] = us_only["rm"]
df_with_us = pd.concat([df, us_only], ignore_index=True)

#-----------------------------------------------
#equaly weighted index returns
#-----------------------------------------------
results_eq = df_with_us.groupby('date').returns_USD.mean()
mean_eq = results_eq.mean()*12
std_eq = results_eq.std()*np.sqrt(12)
sharpe_eq = (mean_eq - 12*df_with_us['rf'].mean())/std_eq

#-----------------------------------------------
# Risk parity index returns
#-----------------------------------------------
result_rp = df_with_us.groupby('country').returns_USD.rolling(60).std().reset_index().rename(columns={'returns_USD': 'std'})
result_rp['date'] = df_with_us['date']
result_rp['weights'] = 1 / result_rp['std']
result_rp['weights'] = result_rp['weights'] / (pd.concat([result_rp.groupby('date').weights.sum().reset_index()['weights']] * 6, ignore_index=True))
result_rp.drop(columns=['level_1', 'std'], inplace=True)
merged_rp = pd.merge(df_with_us, result_rp, on=['date', 'country'])
merged_rp['weighted_return'] = merged_rp['returns_USD'] * merged_rp['weights']
results_rp = merged_rp.groupby('date').weighted_return.sum()
mean_rp = results_rp.mean()*12
std_rp = results_rp.std()*np.sqrt(12)
sharpe_rp = (mean_rp - 12*df_with_us['rf'].mean())/std_rp

#-----------------------------------------------
# Mean variance portfolio returns
#-----------------------------------------------
gamma = 1.0
pivoted_returns = df_with_us.pivot(index='date', columns='country', values='returns_USD')
countries = list(pivoted_returns.columns)
rolling_cov_matrices = pivoted_returns.rolling(window=60).cov().dropna()
rolling_returns_USD = df_with_us.groupby("country").returns_USD.rolling(window=60).mean().reset_index()
rolling_returns_USD['date'] = df_with_us['date']

# Extract covariance matrices as a dictionary
cov_matrices_dict = {
    date: rolling_cov_matrices.loc[date] 
    for date in rolling_cov_matrices.index.get_level_values(0).unique()
}

#for date, cov_matrix in cov_matrices_dict.items():
#    display(cov_matrix.values)

inverted_cov_matrices = {
    date: np.linalg.pinv(cov_matrix.values) for date, cov_matrix in cov_matrices_dict.items()
}

a = {
    date: inv_cov_mat @ (rolling_returns_USD[rolling_returns_USD['date']==date].returns_USD.values - df_with_us[df_with_us['date']==date].rf.values) for date, inv_cov_mat in inverted_cov_matrices.items()
}

weights_mv = []
for date, arr in a.items():
    for idx, val in enumerate(arr):
        weights_mv.append({'date': date, 'weights': val, 'country': countries[idx]})

weights_mv = pd.DataFrame(weights_mv)

merged_mv = pd.merge(df_with_us, weights_mv, on=['date', 'country'])
merged_mv['weighted_return'] = merged_mv['returns_USD'] * merged_mv['weights']
results_mv = merged_mv.groupby('date').weighted_return.sum()
mean_mv = results_mv.mean()*12
std_mv = results_mv.std()*np.sqrt(12)
sharpe_mv = (mean_mv - 12*df_with_us['rf'].mean())/std_mv

summary_unhedged = pd.DataFrame({
    'Portfolio': ['Equal Weight', 'Risk Parity', 'Mean-Variance'],
    'Mean (Annualized)': [mean_eq, mean_rp, mean_mv],
    'Std Dev (Annualized)': [std_eq, std_rp, std_mv],
    'Sharpe Ratio': [sharpe_eq, sharpe_rp, sharpe_mv]
})

display(summary_unhedged)

,Portfolio,Mean (Annualized),Std Dev (Annualized),Sharpe Ratio
0,Equal Weight,0.077003,0.147242,0.420520
1,Risk Parity,0.033416,0.108939,0.168268
2,Mean-Variance,2.208979,1.470027,1.492418


In [48]:
#-----------------------------------------------
# Hedged Index Returns
#-----------------------------------------------

#-----------------------------------------------
#equaly weighted index returns
#-----------------------------------------------
results_eqh = df.groupby('date').hedged_return.mean()
mean_eqh = results_eqh.mean()*12
std_eqh = results_eqh.std()*np.sqrt(12)
sharpe_eqh = (mean_eqh - 12*df['rf'].mean())/std_eqh

#-----------------------------------------------
# Risk parity index returns
#-----------------------------------------------
result_rph = df.groupby('country').hedged_return.rolling(60).std().reset_index().rename(columns={'hedged_return': 'std'})
result_rph['date'] = df['date']
result_rph['weights'] = 1 / result_rph['std']
result_rph['weights'] = result_rph['weights'] / (pd.concat([result_rph.groupby('date').weights.sum().reset_index()['weights']] * 6, ignore_index=True))
result_rph.drop(columns=['level_1', 'std'], inplace=True)
merged_rph = pd.merge(df, result_rph, on=['date', 'country'])
merged_rph['weighted_return'] = merged_rph['hedged_return'] * merged_rph['weights']
results_rph = merged_rph.groupby('date').weighted_return.sum()
mean_rph = results_rph.mean()*12
std_rph = results_rph.std()*np.sqrt(12)
sharpe_rph = (mean_rph - 12*df['rf'].mean())/std_rph

#-----------------------------------------------
# Mean variance portfolio returns
#-----------------------------------------------
gamma = 1.0
pivoted_returns = df.pivot(index='date', columns='country', values='hedged_return')
countries = list(pivoted_returns.columns)
rolling_cov_matrices = pivoted_returns.rolling(window=60).cov().dropna()
rolling_hedged_return = df.groupby("country").hedged_return.rolling(window=60).mean().reset_index()
rolling_hedged_return['date'] = df['date']

# Extract covariance matrices as a dictionary
cov_matrices_dict = {
    date: rolling_cov_matrices.loc[date] 
    for date in rolling_cov_matrices.index.get_level_values(0).unique()
}

#for date, cov_matrix in cov_matrices_dict.items():
#    display(cov_matrix.values)

inverted_cov_matrices = {
    date: np.linalg.pinv(cov_matrix.values) for date, cov_matrix in cov_matrices_dict.items()
}

a = {
    date: inv_cov_mat @ (rolling_hedged_return[rolling_hedged_return['date']==date].hedged_return.values - df[df['date']==date].rf.values) for date, inv_cov_mat in inverted_cov_matrices.items()
}

weights_mvh = []
for date, arr in a.items():
    for idx, val in enumerate(arr):
        weights_mvh.append({'date': date, 'weights': val, 'country': countries[idx]})

weights_mvh = pd.DataFrame(weights_mvh)

merged_mvh = pd.merge(df, weights_mvh, on=['date', 'country'])
merged_mvh['weighted_return'] = merged_mvh['hedged_return'] * merged_mvh['weights']
results_mvh = merged_mvh.groupby('date').weighted_return.sum()
mean_mvh = results_mvh.mean()*12
std_mvh = results_mvh.std()*np.sqrt(12)
sharpe_mvh = (mean_mvh - 12*df['rf'].mean())/std_mvh
# Create a summary table for mean, std dev, and Sharpe ratio of hedged portfolios
summary_hedged = pd.DataFrame({
    'Portfolio': ['Equal Weight', 'Risk Parity', 'Mean-Variance'],
    'Mean (Annualized)': [mean_eqh, mean_rph, mean_mvh],
    'Std Dev (Annualized)': [std_eqh, std_rph, std_mvh],
    'Sharpe Ratio': [sharpe_eqh, sharpe_rph, sharpe_mvh]
})

print("UNHEDGED")
display(summary_unhedged)
print("HEDGED")
display(summary_hedged)


UNHEDGED


,Portfolio,Mean (Annualized),Std Dev (Annualized),Sharpe Ratio
0,Equal Weight,0.077003,0.147242,0.420520
1,Risk Parity,0.033416,0.108939,0.168268
2,Mean-Variance,2.208979,1.470027,1.492418


HEDGED


,Portfolio,Mean (Annualized),Std Dev (Annualized),Sharpe Ratio
0,Equal Weight,0.072768,0.132139,0.436536
1,Risk Parity,0.050825,0.117200,0.304948
2,Mean-Variance,1.553863,1.337623,1.150383
